# Lab 03: Exponential Smoothing
**BSAD 8310: Business Forecasting — University of Nebraska at Omaha**

## Objectives
1. Fit Simple Exponential Smoothing (SES) to monthly retail sales
2. Fit Holt's linear method (with and without damping)
3. Fit Holt-Winters (additive and multiplicative)
4. Use automatic ETS selection (AIC)
5. Compare all models against Lecture 1–2 benchmarks on a 24-month test set
6. Generate and plot 95% prediction intervals

## Dataset
**RSXFS** — US Advance Monthly Retail Trade Survey (Electronics & Appliance Stores),  
retrieved from FRED via `pandas_datareader`. Falls back to a synthetic series if FRED is unavailable.

## Key notation
| Symbol | Meaning |
|--------|---------|
| $\ell_t$ | Level at time $t$ |
| $b_t$ | Trend (slope) at time $t$ |
| $s_t$ | Seasonal component at time $t$ |
| $\alpha$ | Level smoothing parameter |
| $\beta^*$ | Trend smoothing parameter |
| $\gamma$ | Seasonal smoothing parameter |
| $\phi$ | Damping parameter |
| $\hat{y}_{T+h|T}$ | $h$-step-ahead forecast made at time $T$ |

---
## Section 1: Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import warnings
from pathlib import Path

# Statsmodels
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing

warnings.filterwarnings('ignore')
np.random.seed(42)

# --- UNO color palette -------------------------------------------------------
UNO_BLUE  = '#005CA9'
UNO_RED   = '#E41C38'
UNO_GRAY  = '#525252'
UNO_GREEN = '#15803d'
UNO_LBLUE = '#E8F0FA'

# --- Matplotlib defaults -----------------------------------------------------
plt.rcParams.update({
    'figure.dpi': 150,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'axes.prop_cycle': plt.cycler(color=[
        UNO_BLUE, UNO_RED, UNO_GREEN, UNO_GRAY, '#F59E0B', '#7C3AED'
    ]),
    'font.size': 11,
    'axes.titlesize': 13,
    'axes.labelsize': 11,
})

# --- Paths -------------------------------------------------------------------
FIGURES = Path('../Figures')
FIGURES.mkdir(exist_ok=True)

print('Setup complete.')

---
## Section 2: Load Data (FRED RSXFS)

In [ ]:
# Try FRED via pandas_datareader; fall back to AirPassengers-scale synthetic data
try:
    import pandas_datareader.data as web
    raw = web.DataReader('RSXFS', 'fred', start='2000-01-01', end='2023-12-31')
    y = raw['RSXFS'].dropna()
    y.index = pd.PeriodIndex(y.index, freq='M')
    data_source = 'FRED'
except Exception:
    # Fallback: load AirPassengers from statsmodels
    from statsmodels.datasets import get_rdataset
    ap = get_rdataset('AirPassengers').data
    y = pd.Series(
        ap['value'].values,
        index=pd.period_range('1949-01', periods=len(ap), freq='M'),
        name='AirPassengers'
    )
    data_source = 'AirPassengers (fallback)'

print(f'Data source : {data_source}')
print(f'Observations: {len(y)}')
print(f'Period      : {y.index[0]} to {y.index[-1]}')
print(f'\nFirst 5 rows:\n{y.head()}')

# Raw series plot
fig, ax = plt.subplots(figsize=(10, 3.5))
ax.plot(y.to_timestamp(), color=UNO_BLUE, linewidth=1.2)
ax.set_title(f'{y.name} — Raw Series ({data_source})')
ax.set_ylabel('Value')
ax.set_xlabel('')
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_raw.png', bbox_inches='tight')
plt.show()

---
## Section 3: Train / Test Split  (H = 24)

In [ ]:
H = 24  # forecast horizon (months)

train = y.iloc[:-H]
test  = y.iloc[-H:]

print(f'Train: {train.index[0]} to {train.index[-1]}  ({len(train)} obs)')
print(f'Test : {test.index[0]}  to {test.index[-1]}   ({len(test)} obs)')

# Visualise split
fig, ax = plt.subplots(figsize=(10, 3.5))
ax.plot(train.to_timestamp(), color=UNO_BLUE,  linewidth=1.2, label='Train')
ax.plot(test.to_timestamp(),  color=UNO_RED,   linewidth=1.2, label='Test (H=24)')
ax.axvline(train.index[-1].to_timestamp(), color=UNO_GRAY,
           linestyle='--', linewidth=1, label='Split')
ax.set_title('Train / Test Split')
ax.legend()
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_split.png', bbox_inches='tight')
plt.show()

---
## Section 4: Model 1 — Simple Exponential Smoothing (SES)

$$\ell_t = \alpha y_t + (1-\alpha)\ell_{t-1}$$

**Forecast:** $\hat{y}_{T+h|T} = \ell_T$ for all $h$ (flat forecast).

The smoothing parameter $\alpha$ is estimated by minimising the sum of squared one-step-ahead errors.

In [ ]:
# Fit SES on training data
ses_model = SimpleExpSmoothing(train, initialization_method='estimated')
ses_fit   = ses_model.fit(optimized=True)

alpha_hat = ses_fit.params['smoothing_level']
print(f'Estimated alpha : {alpha_hat:.4f}')
print(f'Flat forecast   : {ses_fit.forecast(1).values[0]:.2f}  (same for all h)')

# Forecast H periods ahead
ses_fc = ses_fit.forecast(H)
ses_fc.index = test.index

# Plot smoothed values + forecast
fig, ax = plt.subplots(figsize=(10, 4))
ax.plot(train.to_timestamp(), color=UNO_BLUE, linewidth=1.2, label='Train')
ax.plot(test.to_timestamp(),  color=UNO_GRAY, linewidth=1.2, linestyle='--', label='Actual (test)')
ax.plot(ses_fc.to_timestamp(), color=UNO_RED, linewidth=1.5, label=f'SES forecast (α={alpha_hat:.2f})')
ax.set_title('Simple Exponential Smoothing — Flat Forecast')
ax.legend()
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_ses.png', bbox_inches='tight')
plt.show()

---
## Section 5: Model 2 — Holt's Linear Method

$$\ell_t = \alpha y_t + (1-\alpha)(\ell_{t-1} + b_{t-1})$$
$$b_t = \beta^*(\ell_t - \ell_{t-1}) + (1-\beta^*)b_{t-1}$$

**Forecast:** $\hat{y}_{T+h|T} = \ell_T + h \cdot b_T$

We fit two versions:
- **Holt linear** (no damping)
- **Holt damped** ($0 < \phi < 1$)

In [ ]:
# Holt linear (no damping)
holt_fit = ExponentialSmoothing(
    train,
    trend='add',
    damped_trend=False,
    initialization_method='estimated'
).fit(optimized=True)

# Holt damped
holt_d_fit = ExponentialSmoothing(
    train,
    trend='add',
    damped_trend=True,
    initialization_method='estimated'
).fit(optimized=True)

holt_fc   = holt_fit.forecast(H)
holt_d_fc = holt_d_fit.forecast(H)
holt_fc.index   = test.index
holt_d_fc.index = test.index

print('Holt linear:')
print(f'  alpha = {holt_fit.params["smoothing_level"]:.4f}')
print(f'  beta* = {holt_fit.params["smoothing_trend"]:.4f}')
print()
print('Holt damped:')
print(f'  alpha = {holt_d_fit.params["smoothing_level"]:.4f}')
print(f'  beta* = {holt_d_fit.params["smoothing_trend"]:.4f}')
print(f'  phi   = {holt_d_fit.params["damping_trend"]:.4f}')

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
for ax, fc, label, color in zip(
    axes,
    [holt_fc, holt_d_fc],
    ['Holt linear (no damping)', 'Holt damped'],
    [UNO_GREEN, UNO_RED]
):
    ax.plot(train.to_timestamp(), color=UNO_BLUE, linewidth=1.2, label='Train')
    ax.plot(test.to_timestamp(),  color=UNO_GRAY, linestyle='--', linewidth=1.2, label='Actual')
    ax.plot(fc.to_timestamp(),    color=color,    linewidth=1.5,  label=label)
    ax.set_title(label)
    ax.legend(fontsize=9)
plt.suptitle("Holt's Linear Method", y=1.01)
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_holt.png', bbox_inches='tight')
plt.show()

---
## Section 6: Model 3 — Holt-Winters (Additive and Multiplicative)

Additive update equations ($m = 12$ for monthly data):
$$\ell_t = \alpha(y_t - s_{t-m}) + (1-\alpha)(\ell_{t-1}+b_{t-1})$$
$$b_t = \beta^*(\ell_t - \ell_{t-1}) + (1-\beta^*)b_{t-1}$$
$$s_t = \gamma(y_t - \ell_{t-1} - b_{t-1}) + (1-\gamma)s_{t-m}$$

**Forecast (additive):** $\hat{y}_{T+h|T} = \ell_T + h\,b_T + s_{T+h-m}$

In [ ]:
# Holt-Winters additive
hw_add_fit = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='add',
    seasonal_periods=12,
    initialization_method='estimated'
).fit(optimized=True)

# Holt-Winters multiplicative
hw_mul_fit = ExponentialSmoothing(
    train,
    trend='add',
    seasonal='mul',
    seasonal_periods=12,
    initialization_method='estimated'
).fit(optimized=True)

hw_add_fc = hw_add_fit.forecast(H)
hw_mul_fc = hw_mul_fit.forecast(H)
hw_add_fc.index = test.index
hw_mul_fc.index = test.index

for name, fit in [('Additive', hw_add_fit), ('Multiplicative', hw_mul_fit)]:
    p = fit.params
    print(f'Holt-Winters {name}:')
    print(f'  alpha = {p["smoothing_level"]:.4f}')
    print(f'  beta* = {p["smoothing_trend"]:.4f}')
    print(f'  gamma = {p["smoothing_seasonal"]:.4f}')
    print()

# Plot
fig, axes = plt.subplots(1, 2, figsize=(12, 4), sharey=True)
for ax, fc, label, color in zip(
    axes,
    [hw_add_fc, hw_mul_fc],
    ['Holt-Winters Additive', 'Holt-Winters Multiplicative'],
    [UNO_GREEN, UNO_RED]
):
    ax.plot(train.to_timestamp(), color=UNO_BLUE, linewidth=1.2, label='Train')
    ax.plot(test.to_timestamp(),  color=UNO_GRAY, linestyle='--', linewidth=1.2, label='Actual')
    ax.plot(fc.to_timestamp(),    color=color,    linewidth=1.5,  label=label)
    ax.set_title(label)
    ax.legend(fontsize=9)
plt.suptitle('Holt-Winters Seasonal Methods', y=1.01)
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_hw.png', bbox_inches='tight')
plt.show()

---
## Section 7: Auto-ETS — Select Best Model by AIC

Fit all valid ETS combinations and select the one with the lowest AIC.  
`statsmodels.tsa.holtwinters.ExponentialSmoothing` does not implement full automatic ETS selection natively, so we loop over candidate models and pick the winner.

In [ ]:
candidates = {
    'ETS(A,N,N)': dict(trend=None,  damped_trend=False, seasonal=None),
    'ETS(A,A,N)': dict(trend='add', damped_trend=False, seasonal=None),
    'ETS(A,Ad,N)': dict(trend='add', damped_trend=True,  seasonal=None),
    'ETS(A,A,A)': dict(trend='add', damped_trend=False, seasonal='add'),
    'ETS(A,Ad,A)': dict(trend='add', damped_trend=True,  seasonal='add'),
    'ETS(A,A,M)': dict(trend='add', damped_trend=False, seasonal='mul'),
    'ETS(A,Ad,M)': dict(trend='add', damped_trend=True,  seasonal='mul'),
}

results = {}
for name, kwargs in candidates.items():
    try:
        sp = 12 if kwargs.get('seasonal') else None
        fit = ExponentialSmoothing(
            train,
            seasonal_periods=sp,
            initialization_method='estimated',
            **kwargs
        ).fit(optimized=True)
        results[name] = fit
    except Exception:
        pass

aic_df = pd.DataFrame(
    {'AIC': {k: v.aic for k, v in results.items()}}
).sort_values('AIC')
print(aic_df.to_string())

best_name = aic_df.index[0]
best_fit  = results[best_name]
print(f'\nBest model by AIC: {best_name}  (AIC = {best_fit.aic:.2f})')

best_fc = best_fit.forecast(H)
best_fc.index = test.index

---
## Section 8: Metrics — All Models vs. Benchmarks

In [ ]:
def compute_metrics(actual, forecast):
    """Return dict with RMSE, MAE, MAPE."""
    e = actual.values - np.array(forecast)
    rmse = np.sqrt(np.mean(e**2))
    mae  = np.mean(np.abs(e))
    mape = np.mean(np.abs(e / actual.values)) * 100
    return {'RMSE': rmse, 'MAE': mae, 'MAPE (%)': mape}

# --- Lecture 1 benchmark forecasts (re-fit here for comparison) --------------
naive_fc     = np.repeat(train.iloc[-1], H)             # naïve
snaive_fc    = np.tile(train.iloc[-12:].values, 2)[:H]  # seasonal naïve
mean_fc      = np.repeat(train.mean(), H)               # historical mean

# --- Lecture 2 AR(2) proxy (simple AR via OLS) -------------------------------
from statsmodels.tsa.ar_model import AutoReg
ar_fit = AutoReg(train, lags=2, old_names=False).fit()
ar_fc  = ar_fit.forecast(steps=H)

# --- All model forecasts ─────────────────────────────────────────────────────
forecasts = {
    'Naïve'          : naive_fc,
    'Seasonal naïve' : snaive_fc,
    'Mean'           : mean_fc,
    'AR(2)'          : ar_fc.values,
    'SES'            : ses_fc.values,
    'Holt linear'    : holt_fc.values,
    'Holt damped'    : holt_d_fc.values,
    'HW additive'    : hw_add_fc.values,
    'HW multiplicative': hw_mul_fc.values,
    f'Auto-ETS ({best_name})': best_fc.values,
}

rows = []
for name, fc in forecasts.items():
    row = compute_metrics(test, fc)
    row['Model'] = name
    rows.append(row)

metrics_df = pd.DataFrame(rows).set_index('Model')[['RMSE', 'MAE', 'MAPE (%)']]
metrics_df = metrics_df.sort_values('RMSE')
print(metrics_df.round(2).to_string())

---
## Section 9: Forecast Plot with 95% Prediction Intervals

We compute prediction intervals for the **best additive-error ETS model**  
(additive-error models support closed-form Gaussian PIs).

In [ ]:
# Use HW additive for PI (additive error → closed-form Gaussian PI)
pi_fit = hw_add_fit

# statsmodels simulate-based PI
sim = pi_fit.simulate(
    nsimulations=H,
    repetitions=5000,
    error='add',
    random_errors='bootstrap'
)
pi_lo = np.percentile(sim, 2.5,  axis=1)
pi_hi = np.percentile(sim, 97.5, axis=1)

# ── All-model summary plot ───────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(12, 5))

# Training data (last 48 months for clarity)
tail = train.iloc[-48:]
ax.plot(tail.to_timestamp(), color=UNO_BLUE, linewidth=1.4, label='Training data', zorder=5)
ax.plot(test.to_timestamp(), color=UNO_GRAY,  linewidth=1.4, linestyle='--', label='Actual', zorder=5)

# Benchmark: seasonal naïve
ax.plot(test.to_timestamp(), snaive_fc, color=UNO_GRAY, linewidth=1,
        linestyle=':', label='Seasonal naïve', alpha=0.8)

# Best ETS
ax.plot(best_fc.to_timestamp(), color=UNO_RED, linewidth=2,
        label=f'Auto-ETS: {best_name}', zorder=6)

# HW additive + PI
hw_ts = hw_add_fc.to_timestamp()
ax.plot(hw_ts, hw_add_fc.values, color=UNO_GREEN, linewidth=1.5,
        label='HW Additive + 95% PI')
ax.fill_between(hw_ts, pi_lo, pi_hi, color=UNO_GREEN, alpha=0.15)

ax.axvline(train.index[-1].to_timestamp(), color=UNO_GRAY,
           linestyle='--', linewidth=0.8)
ax.set_title('Exponential Smoothing Forecasts vs.  Actual')
ax.set_ylabel('Value')
ax.legend(fontsize=9, loc='upper left')
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_forecasts.png', bbox_inches='tight')
plt.show()

# ── Metrics bar chart ────────────────────────────────────────────────────────
fig, ax = plt.subplots(figsize=(10, 4))
colors = [UNO_GRAY]*3 + [UNO_GRAY] + [UNO_BLUE]*5 + [UNO_RED]
rmse_vals = metrics_df['RMSE']
bars = ax.barh(rmse_vals.index, rmse_vals.values,
               color=colors[:len(rmse_vals)], edgecolor='white')
ax.set_xlabel('RMSE (lower is better)')
ax.set_title('Model Comparison: RMSE on 24-Month Test Set')
ax.invert_yaxis()
for bar, val in zip(bars, rmse_vals.values):
    ax.text(val + 0.5, bar.get_y() + bar.get_height()/2,
            f'{val:.1f}', va='center', fontsize=9)
plt.tight_layout()
plt.savefig(FIGURES / 'lecture03_metrics.png', bbox_inches='tight')
plt.show()

print('\nFigures saved to Figures/')